In [16]:
import logging
#log_level = logging.INFO
# log_level = logging.DEBUG
log_level = 25 # ANALYSIS_LEVEL
logging.basicConfig(level=log_level)
logger = logging.getLogger(__name__)
logger.setLevel(log_level)

import os
from dotenv import load_dotenv
load_dotenv()
secret_name = "OPENAI_API_KEY_CEMAD"
openai_api_key = os.getenv(secret_name)
key = os.getenv('DECRYPTION_KEY_CEMAD')


In [17]:
import os
import pandas as pd
from openai import OpenAI

from regulations_rag.embeddings import  EmbeddingParameters
from regulations_rag.rerank import RerankAlgos
from regulations_rag.corpus_chat import ChatParameters

import importlib
import cemad_rag.cemad_corpus_index
importlib.reload(cemad_rag.cemad_corpus_index)
from cemad_rag.cemad_corpus_index import CEMADCorpusIndex

import cemad_rag.corpus_chat_cemad
importlib.reload(cemad_rag.corpus_chat_cemad)
from cemad_rag.corpus_chat_cemad import CorpusChatCEMAD

from IPython.display import Markdown, display


In [18]:
chat_parameters = ChatParameters(chat_model = "gpt-4o", api_key = openai_api_key, temperature = 0, max_tokens = 4096, token_limit_when_truncating_message_queue = 3500)

embedding_parameters = EmbeddingParameters("text-embedding-3-large", 1024)
corpus_index = CEMADCorpusIndex(key)

rerank_algo  = RerankAlgos.LLM
if rerank_algo == RerankAlgos.LLM:
    rerank_algo.params["openai_client"] = chat_parameters.openai_client
    rerank_algo.params["model_to_use"] = chat_parameters.model
    rerank_algo.params["user_type"] = corpus_index.user_type
    rerank_algo.params["corpus_description"] = corpus_index.corpus_description
    rerank_algo.params["final_token_cap"] = 5000 # can go large with the new models


chat = CorpusChatCEMAD(
                    embedding_parameters = embedding_parameters, 
                    chat_parameters = chat_parameters, 
                    corpus_index = corpus_index,
                    rerank_algo = RerankAlgos.LLM,   
                    user_name_for_logging = 'test_user')

chat.strict_rag = False


In [19]:
#  May minors purchase foreign exchange
user_content = "Can I use my credit card for online purchases?"
user_content = "Can I open a non-resident account for a CMA resident from another CMA country"
user_content = "What are the foreign currency limits for children?"
user_content = "what do the regulations say about travel?"
user_content = "How much money can an individual take offshore in a year?"
#user_content = "What do the regs say about crypto?"
#user_content = "Can an individual can take more than R1 million offshore in a year for travel?"

# user_content = "What if I need more that R1 million for travel?"
# user_content = "I need more than R1 million for travel this year. What do I do?"
user_content = "What is a travel allowance?"
user_content = "Can I buy property overseas using my credit card?"
#user_content = "Can I purchase property in a foreign country using a credit card?"
chat.reset_conversation_history()

chat.user_provides_input(user_content)
#chat.user_provides_input(user_content)
display(Markdown((chat.messages_intermediate[-1])['content']))

ANALYSIS:regulations_rag.corpus_chat:test_user: Can I buy property overseas using my credit card?
ANALYSIS:regulations_rag.corpus_chat:test_user: performing RAG ...
ANALYSIS:regulations_rag.corpus_chat:test_user: RAG returned the response.


No, you cannot use a South African credit card to buy property overseas. The use of South African credit, debit, and virtual cards is not permitted for funding foreign currency accounts or for funding international trading accounts abroad. Transactions for foreign investments must be conducted through the conversion of Rand into foreign currency by an Authorised Dealer and transferred via the banking system as an Electronic Funds Transfer. 

Reference: 

Section B.2(B)(i) from Currency and Exchange Control Manual for Authorised Dealers: 

B.2 Capital transfers

&nbsp;&nbsp;&nbsp;&nbsp;(B) Private individuals resident in South Africa

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(i) Foreign investments by private individuals (natural persons) resident in South Africa

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(a) Authorised Dealers may allow the transfer, as a foreign capital allowance, of up to a total amount of R10 million per calendar year per private individual who is a taxpayer in good standing and is 18 years and older, for investment purposes abroad. The funds to be transferred must be converted to foreign currency by the Authorised Dealer and may also be held in a resident foreign currency account in the name of the resident with any Authorised Dealer.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(b) Authorised Dealers are advised that a valid green bar-coded South African identity document or a Smart identity document card is the only acceptable document proving residency in South Africa.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(c) Prior to authorising the transaction, Authorised Dealers must ensure that their client is acquainted with the declaration contained in the integrated form.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(d) In terms of the SARS Tax Compliance Status (TCS) system, a TCS PIN letter will be issued to the taxpayer that will contain the tax number and TCS PIN. Authorised Dealers must use the TCS PIN to verify the taxpayer's tax compliance status via SARS eFiling prior to effecting any transfers. Authorised Dealers must ensure that the amount to be transferred does not exceed the amount approved by SARS. Authorised Dealers should note that the TCS PIN can expire and should the Authorised Dealers find that the TCS PIN has indeed expired, then the Authorised Dealer must insist on a new TCS PIN to verify the taxpayer's tax compliance status.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(e) In terms of the SARS Tax Compliance Status System introduced on 2016-04-18, a tax compliance status PIN letter will be issued to the taxpayer that will contain the tax number and PIN. Authorised Dealers must use the PIN to verify the taxpayer's tax compliance status via SARS eFiling prior to effecting any transfers.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(f) Authorised Dealers must retain a printed tax compliance status verification result for a period of five years for inspection purposes.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(g) Private individuals who do not have a tax reference number will have to register at their local SARS branch.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(h) Authorised Dealers must bring to the attention of their clients that they may not enter into any transactions whereby capital or the right to capital will be directly or indirectly exported from South Africa (e.g. may not enter into a foreign commitment with recourse to South Africa). However, private individuals may raise loans abroad to finance the acquisition of foreign assets without recourse to South Africa.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(i) Resident individuals with authorised foreign assets may invest in South Africa, provided that where South African assets are acquired through an offshore structure (loop structure), the investment is reported to an Authorised Dealer as and when the transaction(s) is finalised as well as the submission of an annual progress report to the Financial Surveillance Department via an Authorised Dealer. The aforementioned party also has to view an independent auditor's written confirmation or suitable documentary evidence verifying that such transaction(s) is concluded on an arm's length basis, for a fair and market related price.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(j) Upon completion of the transaction in (i) above the Authorised Dealer must submit a report to the Financial Surveillance Department which should, inter alia, include the name(s) of the South African affiliated foreign investor(s), a description of the assets to be acquired (including inward foreign loans, the acquisition of shares and the acquisition of property), the name of the South African target investment company, if applicable and the date of the acquisition as well as the actual foreign currency amount introduced including a transaction reference number.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(k) In addition, all inward loans from South African affiliated foreign investors must comply with the directives issued in section I.3(B) of the Authorised Dealer Manual.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(l) Existing unauthorised loop structures (i.e. created by individuals prior to 2021-01-01) and/or unauthorised loop structures where the 40 per cent shareholding threshold was exceeded, must still be regularised with the Financial Surveillance Department.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(m) The Financial Surveillance Department will consider applications by private individuals who wish to invest in excess of the R10 million foreign capital allowance limit, in different asset classes. Such investments, if approved, could be facilitated via a foreign domiciled and registered trust. This dispensation would also apply to private individuals who have existing authorised foreign assets, irrespective of the value thereof. In terms of the TCS system, a TCS PIN letter will be issued to the taxpayer that will contain the tax number and TCS PIN. Authorised Dealers must use the TCS PINto verify the taxpayer's tax compliance status via SARS eFiling prior to effecting any transfers. Authorised Dealers should note that the TCS PIN can expire and should the Authorised Dealers find that the TCS PIN has indeed expired, then the Authorised Dealer must insist on a new TCS PIN to verify the taxpayer's tax compliance.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(n) Foreign currency accounts may be opened for private individuals (natural persons). Authorised Dealers may allow withdrawals from these accounts through the use of any authorised banking product. Funds withdrawn from these accounts may also be converted to Rand.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(o) Private individuals may, as part of their single discretionary allowance and/or foreign capital allowance, export multi-listed domestic securities to a foreign securities register in a jurisdiction where such securities are listed, subject to tax compliance and reporting to the Financial Surveillance Department via a Central Securities Depository Participant, in conjunction with an Authorised Dealer.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(p) Since these transactions will not result in actual flow of funds from South Africa, the process thereof as well as the reporting must take place as outlined hereunder:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(aa) An applicant must furnish the following information through an Authorised Dealer, at the time of submitting a request for a confirmation letter and/or approval letter to the Financial Surveillance Department:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(1) the full names and identity number of the applicant;

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(2) the name of the company whose securities are exported as well as number of securities and the market value thereof; and

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(3) domicilium and name of the target foreign register on which those securities are listed.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(bb) In this regard, the confirmation letter and/or approval letter from the Financial Surveillance Department must be presented by the applicant to the relevant South African Central Securities Depository Participant to effect the transaction.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(cc) Central Securities Depository Participants may, in conjunction with an Authorised Dealer, allow the transfer of domestic listed securities abroad, up to a total market value of R1 million per calendar year in terms of the single discretionary allowance for private individuals, without the requirement to obtain a TCS PIN letter, provided a confirmation letter from the Financial Surveillance Department is viewed.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(dd) Central Securities Depository Participants may, in conjunction with an Authorised Dealer, also allow the transfer of domestic listed securities of up to a total market value of R10 million per calendar year in terms of the foreign capital allowance, provided that a TCS PIN letter is obtained as well as a confirmation letter from the Financial Surveillance Department is viewed.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(ee) Private individuals who export securities with a market value of more than R10 million are subject to a more stringent verification process by SARS as well as an approval process from the Financial Surveillance Department. Such transfers will trigger a risk management test that will, inter alia, include verification of the tax status and the source of funds, as well as risk assess the private individual in terms of the anti-money laundering and countering terror financing requirements, as prescribed in the Financial Intelligence Centre Act, 2001 (Act No. 38 of 2001).

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(q) Private individuals may only fund online international trading accounts at registered brokers in terms of the single discretionary and/or foreign capital allowance, i.e. the Authorised Dealer concerned must convert the Rand into foreign currency and transfer such funds via the banking system as an Electronic Funds Transfer to a foreign bank account or the funds can be deposited in a foreign currency account at an Authorised Dealer.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(r) No South African debit, credit and virtual card may, however, be used to fund a foreign currency account at an Authorised Dealer or a bank abroad, nor may international trading accounts of private individuals be funded using South African credit, debit and virtual card transfers. Online international trading accounts, inter alia include trading global currencies against each other, trading a contract for difference, trading in foreign stocks, trading commodities including crypto assets and/or trading foreign indices using an online trading platform of the broker concerned.  



In [14]:
from cemad_rag.path_suggest_alternatives import PathSuggestAlternatives
path_suggest_alternatives = PathSuggestAlternatives(chat_parameters = chat_parameters, corpus_index = corpus_index, embedding_parameters = embedding_parameters, rerank_algo = rerank_algo)
user_content = "Can I buy property overseas using my credit card?"
path_suggest_alternatives.suggest_alternative_questions(message_history = [], current_user_message = {'role': 'user', 'content': user_content})



ANALYSIS:cemad_rag.path_suggest_alternatives:Suggesting alternative questions


{'role': 'assistant',
 'content': 'The system has suggested the following alternative questions: \n\nCan I purchase property in a foreign country using a credit card?\nAre there restrictions on using a credit card to buy property in a foreign country?\nWhat are the regulations for South Africans using credit cards to purchase property abroad?',
 'assistant_response': AlternativeQuestionResponse(alternatives=['Can I purchase property in a foreign country using a credit card?', 'Are there restrictions on using a credit card to buy property in a foreign country?', 'What are the regulations for South Africans using credit cards to purchase property abroad?'])}

In [10]:
user_content = "Do children get a travel allowance?"
chat.user_provides_input(user_content)
display(Markdown(chat.messages_intermediate[-1]['content']))

ANALYSIS:regulations_rag.corpus_chat:test_user: Do children get a travel allowance?
ANALYSIS:regulations_rag.corpus_chat:test_user: performing RAG ...
ANALYSIS:regulations_rag.corpus_chat:test_user: RAG returned the response.


Yes, children who are residents and under the age of 18 years may be accorded a travel allowance not exceeding an amount of R200,000 per calendar year. 

Reference: 

Section B.4(B)(i) from Currency and Exchange Control Manual for Authorised Dealers: 

B.4 Single discretionary allowance and other miscellaneous payments for private individuals

&nbsp;&nbsp;&nbsp;&nbsp;(B) Travel allowances

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(i) General

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(a) There is no limit on the amount of Rand that may be spent on travelling within the CMA.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(b) Residents (natural persons) who are under the age of 18 years may not be accorded single discretionary allowance as outlined in subsection (A)(i) above, but may be accorded a travel allowance not exceeding an amount of R200 000 per calendar year.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(c) In addition, up to R25 000 in cash, per person, may be taken when proceeding on visits outside the CMA, to meet travellers' immediate needs on return to South Africa.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(d) When according travel allowances to persons who do not conduct accounts in their books, Authorised Dealers must record such persons' names, nationalities, residential addresses and telephone numbers to facilitate communication if needed.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(e) Visiting artistes, entertainers, sportspersons and other similar professionals may not be accorded the facilities outlined in this section of the Authorised Dealer Manual.  

Section B.4(B)(iv)(a) from Currency and Exchange Control Manual for Authorised Dealers: 

B.4 Single discretionary allowance and other miscellaneous payments for private individuals

&nbsp;&nbsp;&nbsp;&nbsp;(B) Travel allowances

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(iv) Eligible parties

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(a) Residents

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(aa) A travel allowance within the single discretionary allowance limit of R1 million per calendar year may be availed of by residents (natural persons) (excluding persons proceeding abroad to study) who are 18 years and older.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(bb) Residents (natural persons) who are under the age of 18 years may avail of a travel allowance not exceeding an amount of R200 000 per calendar year.  



In [6]:
user_content = "How can an individual apply for a travel allowance above R1 million for the year?"
chat.user_provides_input(user_content)
display(Markdown(chat.messages_intermediate[-1]['content']))

ANALYSIS:regulations_rag.corpus_chat:test_user: How can an individual apply for a travel allowance above R1 million for the year?
ANALYSIS:regulations_rag.corpus_chat:test_user: performing RAG ...
ANALYSIS:regulations_rag.corpus_chat:test_user: RAG returned a NoAnswerResponse. Trying without the retrieved data ...
ANALYSIS:regulations_rag.corpus_chat:test_user: query_no_rag_data() returned the response.


No Answer

In [6]:
display(Markdown(chat.messages_intermediate[-1]['content']))

NOTE: The following answer is provided without references and should therefore be treated with caution.

If you need to transfer more than the R11 million limit (R10 million foreign capital allowance plus R1 million single discretionary allowance) in a calendar year, you can apply to the South African Reserve Bank's Financial Surveillance Department for approval. This application will require a tax compliance status verification and may involve a more stringent assessment process, including verification of the source of funds and compliance with anti-money laundering regulations.

In [7]:
#user_content = "What documentation do they require?"
user_content = "What documentation is required?"
chat.user_provides_input(user_content)

# testing = True
# path = "ANSWER:"
# api_response = "Yes, a foreigner can buy property in South Africa. However, if they require local financial assistance for the acquisition, certain conditions apply. Specifically, the 1:1 ratio will apply for financial assistance unless the non-resident is living and working in South Africa, in which case normal lending criteria apply. Additionally, any financial assistance must be secured by an unencumbered Rand deposit or Rand-based asset of equal or higher value, and the facility may not exceed 100% of the Rand value of funds introduced from abroad and invested locally. Facilities for property acquisition cannot be increased based on property revaluation.Reference: 1"
# manual_responses_for_testing = [path + api_response]

# chat.user_provides_input(user_content, testing = testing, manual_responses_for_testing = manual_responses_for_testing)

ANALYSIS:regulations_rag.corpus_chat:test_user question: What documentation is required?
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
DEV:regulations_rag.corpus_chat:#################   Similarity Search       #################
INFO:regulations_rag.corpus_chat:Found a potentially relevant workflow: documentation
DEV:regulations_rag.corpus_index:--   No relevant definitions found
DEV:regulations_rag.corpus_index:Selecting the top 15 items based on cosine-similarity score
DEV:regulations_rag.corpus_index:0.3339:                CEMAD:              B.12(A):        question: What documentation is required to conduct a merchanting trade?
DEV:regulations_rag.corpus_index:0.3547:                CEMAD:               B.8(C):        question: What documentation is required for making subsequent payments for a tour?
DEV:regulations_rag.corpus_index:0.3617:                CEMAD:         D.1(C)(i)(d):        question: What documentation is required for import-

In [19]:
llm_reply

'ANSWER: An individual can take up to a total amount of R10 million per calendar year offshore as a foreign capital allowance for investment purposes abroad. Additionally, within the single discretionary allowance, an individual can take up to R1 million per calendar year offshore. Therefore, the total amount an individual can take offshore in a year is R11 million.\n\nReference: 1, 2'

In [7]:
from IPython.display import Markdown, display

# m = chat.corpus.get_text("Forgotten", "1")
# display(Markdown(m))    

d = chat.corpus.get_text("CEMAD", "B.10(A)(i)")
d
#print(chat.corpus.get_text("CovidLocation", "1"))
#print(chat.corpus.get_text("Consent", "7.1.3").strip())
#print(chat.corpus.get_text("GDPR", "8").strip())

'B.10 Insurance and pensions\n\n&nbsp;&nbsp;&nbsp;&nbsp;(A) Introduction\n\n&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(i) Only those branches of the Authorised Dealers referred to in subsection (K) below may effect payments in foreign currency in terms of this section of the Authorised Dealer Manual.'

In [9]:
wf, dfn, section = chat.similarity_search("Can a company use its credit card for online purchases?")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
DEV:regulations_rag.corpus_chat:#################   Similarity Search       #################
DEV:regulations_rag.corpus_chat:No relevant workflow found
DEV:regulations_rag.corpus_index:--   No relevant definitions found
DEV:regulations_rag.corpus_index:Selecting the top 15 items based on cosine-similarity score
DEV:regulations_rag.corpus_index:0.2267:                CEMAD:              B.16(E):        question: Can I use my credit card for online purchases?
DEV:regulations_rag.corpus_index:--   Relevant sections found
DEV:regulations_rag.rerank:Re-ranking using LLM
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
DEV:regulations_rag.rerank:--   results requested by LLM filter
DEV:regulations_rag.rerank:B.16(E)


In [8]:
section

,section_reference,text,source,embedding,document,cosine_distance,regulation_text,token_count
0,B.16(E),Can I use my credit card for online purchases?,question,"[-0.06440070271492004, 0.03733660280704498, -0...",CEMAD,0.113945,B.16 Credit and/or debit cards\n (E) Miscel...,279


In [5]:
print(chat.create_system_message(3))

You are answering questions about the General Data Protection Regulation (GDPR) for a Controller based only on the reference extracts provided. You have 3 options:
1) Answer the question. Preface an answer with the tag 'ANSWER:'. All referenced extracts must be quoted at the end of the answer, not in the body, by number, in a comma separated list starting after the keyword 'Reference: '. Do not include the word Extract, only provide the number(s).
2) Request additional documentation. If, in the body of the extract(s) provided, there is a reference to another section that is directly relevant and not already provided, respond with the word 'SECTION:' followed by 'Extract extract_number, Reference section_reference' - for example SECTION: Extract 1, Reference Article 98.
3) State 'NONE:' and nothing else in all other cases



In [ ]:
# BCR
user_content = "What are binding corporate rules?"
user_content = "How do binding corporate rules become approved?"
user_content = "What requirements must binding corporate rules meet?"
user_content = "What details must be spelled out in binding corporate rules?"
user_content = "What rights do individuals have under binding corporate rules?"
user_content = "Who is liable if binding corporate rules are breached?"
user_content = "How are individuals informed about their rights under binding corporate rules?"
user_content = "What is the role of a data protection officer within a group adopting binding corporate rules?"
user_content = "What mechanisms ensure compliance with binding corporate rules?"
user_content = "How is the effectiveness of binding corporate rules verified?"
user_content = "What happens if local laws conflict with the standards of binding corporate rules?"
user_content = "What kind of training is required for employees with access to personal data under binding corporate rules?"
user_content = "What is the process for reporting changes in binding corporate rules?"

# Decision Making
user_content = "What rights do individuals have regarding automated decision-making and profiling?"
user_content = "What does it mean for a decision to have a legal effect on a person?"
user_content = "Under what conditions can an individual be subject to automated decision-making?"
user_content = "What safeguards must be in place when automated decisions are necessary for contracts?"
user_content = "Can special categories of personal data be used in automated decision-making?"

#DPIA
user_content = "When must you conduct a data protection impact assessment?"
user_content = "Who should you consult when carrying out a data protection impact assessment?"
user_content = " What triggers the requirement for a data protection impact assessment?"
user_content = "How do supervisory authorities contribute to the DPIA process?"
user_content = "How do supervisory authorities contribute to the data protection impact assessment process?"
user_content ="What content is required in a data protection impact assessment?"
user_content ="What content is required in a DPIA?"
user_content = "Should individuals' views be considered in the assessment process?"
user_content = "How often should you review the data protection impact assessment?"

#DPO
user_content = "When must you designate a data protection officer?"
user_content = "Can multiple organisations share a DPO?"
user_content = "What criteria must a data protection officer meet?"
user_content = "Can a data protection officer be a contractor?"
user_content = "How should the contact details of the DPO be shared?"
user_content = "What types of processing activities require you to appoint a data protection officer?"
user_content = "How does the scale of data processing affect the requirement to appoint a data protection officer?"
user_content = "What is large scale?"
user_content = "What is the difference between a data protection officer and a controller?"

# intl_transfer
user_content = "What are the conditions for transferring personal data to another country or international organisation?"
user_content = "When is an individual's consent required for data transfer?"
user_content = "How does a contract affect the transfer of personal data?"
user_content = "What constitutes an important reason of public interest for transferring personal data?"
user_content = "When can data be transferred for the establishment, exercise, or defence of legal claims?"
user_content = "Under what circumstances can personal data be transferred to protect someone's vital interests?"
user_content = "What are the rules for transferring data from public registers?"
user_content = "What are the requirements for a non-repetitive transfer based on compelling legitimate interests?"
user_content = "When are public authorities exempt from certain transfer conditions?"
user_content = "How does the law recognise public interest for data transfers?"
user_content = "What documentation is required for assessing data transfers?"

# lead_sa

# data breach
user_content = "What action should you take if a personal data breach occurs?"
user_content = "How soon must you notify the appropriate authority when a data breach is discovered?"
user_content = "Is there any situation where you might not need to notify a supervisory authority about a data breach?"
user_content = "What information must you include when reporting a data breach?"
user_content = "What should be done if all details about a breach can't be reported immediately?"
user_content = "Why is it important to document all personal data breaches and the response to them?"

user_content = "When should you inform individuals about a personal data breach?"
user_content = "What information should be included when notifying an individual about a personal data breach?"
user_content = "Under what conditions can you avoid informing individuals about a personal data breach?"
user_content = "What happens if you don't voluntarily notify individuals about a personal data breach?"


# portability
user_content = "What is the right to data portability?"
user_content = "What does data portability mean for the transfer of data between controllers?"
user_content = "Is the right to data portability unlimited?"

# transparency
user_content = "What information must be provided to individuals when collecting their personal data?"
user_content = "What details about the data controller need to be disclosed during data collection?"
user_content = "What are the required disclosures about the use and processing of personal data?"
user_content = "When is it necessary to inform individuals about the transfer of their personal data to third countries or international organisations?"
user_content = "What rights must individuals be informed about concerning their personal data?"
user_content = "How does the requirement for providing information change if the purpose of data processing changes?"
user_content = "What information must be provided to individuals when their data is collected indirectly?"
user_content = "Within what timeframe must information be provided to the data subject when their data is collected indirectly?"
# user_content = "What actions must be taken if the purpose of data processing changes after initial collection?"
# user_content = "Are there any circumstances under which the obligation to provide information to data subjects does not apply?"
# user_content = "How are data subjects informed about their rights concerning their personal data?"


# Codes
user_content = "Who is encouraged to create codes of conduct for GDPR compliance?"
user_content = "What purposes do codes of conduct serve?"
user_content = "Can codes of conduct address the protection of children's data?"
user_content = "How can codes of conduct assist with personal data breaches?"
user_content = "Can organisations outside the EU adhere to codes of conduct?"
user_content = "What requirements must a code of conduct meet for approval?"
# user_content = "Who approves codes of conduct for local purposes?"
# user_content = "How are codes of conduct evaluated for processing activities across multiple Member States?"
# user_content = "What role does the Commission play in the validity of codes of conduct?"
# user_content = "How is the public informed about approved codes of conduct?"
# user_content = "Where can one find a register of approved codes of conduct?"
# user_content = "Who can monitor compliance with a code of conduct?"
# user_content = "What qualifications must a body meet to monitor compliance with a code of conduct?"
user_content = "How does a body become accredited to monitor a code of conduct?"
# user_content = "What responsibilities does an accredited body have when a code of conduct is breached?"
# user_content = "What actions can an accredited monitoring body take if a code of conduct is infringed?"
# user_content = "Under what circumstances can an accreditation be revoked?"

#user_content = "Can you give me a worfklow for the approval of a code of conduct?"

# territorial scope
user_content = "When does GDPR apply to non-EU entities?"
user_content = "I am not based in the EU. Does GDPR apply to me?"

# video
user_content = "Is video surveillance regulated under gdpr?"

# Covid Health
user_content = "What specific purposes allow healthcare data to be processed without explicit consent?"
user_content = "What are the permissible legal bases for processing health data for scientific research?"

# covid location
user_content = "What types of data should not be collected by contact tracing applications?"

# Concent
user_content = "Can children consent to data collection?"

# Forgotten
user_content = "What is the right to erasure and when does it apply?"


In [1]:
from gdpr_rag.documents.dpo import DPO
from gdpr_rag.documents.consent import Consent
dpo = DPO()
consent = Consent()

In [23]:
file = "./inputs/index/video.parquet"
df = pd.read_parquet(file)
df

,section_reference,text,source,embedding,document
0,2.1,gAAAAABmUH9YDpyn75A6P5UBv2tn3enDzLAPpQYXhXxsuk...,summary,"[-0.05341560021042824, 0.0171151515096426, -0....",Video
1,2.2,gAAAAABmUH9YEm2qVPRmHM-J5GQxVp8K7JCYlxzpbBAt6h...,summary,"[-0.05042532831430435, -0.034690551459789276, ...",Video
2,2.3,gAAAAABmUH9Yt_9fPYROaTi2ZIdE-IGpQpFtdHQ1tI6IpM...,summary,"[-0.05431677773594856, 0.031087687239050865, -...",Video
3,3,gAAAAABmUH9YI-aMX31OE0KoFJtf5__giOrAD1XTEiVt1q...,summary,"[0.011479574255645275, -0.04576420783996582, -...",Video
4,3.1.1,gAAAAABmUH9YMHnezbumcNuKAdDdVAelDbD5cu0VuRpGAz...,summary,"[0.024606412276625633, -0.012803575955331326, ...",Video
...,...,...,...,...,...
123,9.3.2,gAAAAABmUH9YxhYBbjnCjhfyzJSnOqOND6Egd3ErKrbHPX...,question,"[-0.004779711831361055, 0.0069802324287593365,...",Video
124,9.3.2,gAAAAABmUH9YaTg645WLz3LZdFQ_zBII1t0IXvpW96hTbw...,question,"[-0.045810651034116745, 0.048128221184015274, ...",Video
125,9.3.2,gAAAAABmUH9YSxCyP9WC7aFGTLMJSCrcL84pCwxTkSPlhz...,question,"[-0.02516673319041729, -0.024208365008234978, ...",Video
126,10,gAAAAABmUH9YJjSfd9EKsOg7w3xxvlvNGZ5J0T1SWUEaug...,question,"[-0.039777971804142, -0.014415937475860119, -0...",Video
